<a href="https://colab.research.google.com/github/Petint/AI-and-DS/blob/main/ai_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI-2

In [12]:
# Import all necessary
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2 as cv

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D
from keras.layers import BatchNormalization, Activation, Dropout
from keras.preprocessing.image import ImageDataGenerator

# Connect to drive

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Dara prep

In [14]:
path = '/content/drive/MyDrive/cropped_images/'
input_shape = (96, 96, 3)
x = []
y = []

for folder in os.listdir(path):
  for images in os.listdir(os.path.join(path, folder)):
    imagepath = os.path.join(path, folder, images)
    image = cv.imread(imagepath)
    image = cv.resize(image, (input_shape[1], input_shape[0]))
    label = folder.replace('_', ' ')
    x.append(image)
    y.append(label)

# Norm

In [15]:
x = np.array(x) / 255.0
y = np.array(y)
# Herbalizer
names = np.unique(y)
print(names)
print(y[60])
mlb = LabelBinarizer()
y = mlb.fit_transform(y)
print(y[60])


['chris evans' 'chris hemsworth' 'mark ruffalo' 'robert downey jr'
 'scarlett johansson']
robert downey jr
[0 0 0 1 0]


# Train - test - validate

In [16]:
x, x_test, y, y_test, =train_test_split(x, y, test_size=0.1, random_state=1)
x_train, x_val, y_train, y_val, =train_test_split(x, y, test_size=0.2, random_state=1)

print(x.shape, x_test.shape, y.shape, y_test.shape)
print(x_train.shape, x_val.shape, y_train.shape, y_val.shape)

(246, 96, 96, 3) (28, 96, 96, 3) (246, 5) (28, 5)
(196, 96, 96, 3) (50, 96, 96, 3) (196, 5) (50, 5)


# Datagen

In [17]:
datagen = ImageDataGenerator(
    rotation_range=45,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2,
)

# Make Shrek have layers

In [18]:
onion = Sequential()
# Layer 1
onion.add(Conv2D(32, 3, input_shape=input_shape, activation='relu'))
onion.add(MaxPooling2D(3))
onion.add(Dropout(0.25))
# Layer 2
onion.add(Conv2D(64, 3, padding='same', activation='relu'))
onion.add(Conv2D(64, 3, padding='same', activation='relu'))
onion.add(MaxPooling2D())
onion.add(Dropout(0.25))
# Layer 3
onion.add(Conv2D(128, 3, padding='same', activation='relu'))
onion.add(Conv2D(128, 2, padding='same', activation='relu'))
onion.add(MaxPooling2D())
onion.add(Dropout(0.25))
# Layer 4
onion.add(Flatten())
onion.add(Dense(1024, activation='relu'))
# Layer 5
onion.add(Dense(len(mlb.classes_), activation='softmax'))

onion.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 94, 94, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 31, 31, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 31, 31, 64)        18496     
                                                                 
 conv2d_2 (Conv2D)           (None, 31, 31, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 15, 15, 64)       0         
 2D)                                                    

# Fti moddel